In [ ]:
# Read interferograms directly from s3 location
import s3fs
import geoviews as gv
import hvplot.xarray
import xarray as xr
import numpy as np
import geopandas as gpd
import hvplot.pandas


In [ ]:
fs = s3fs.S3FileSystem()
folders = fs.ls('uturuncu-insar')

In [ ]:
folders

In [ ]:
# Full directory listing (NOTE A76 missing geocoded files!) 
# NOTE: not updated if files synced to S3. Need to restart FileSystem (or clear cache somehow...)
#fs.ls('uturuncu-insar/2019-09/A76/int-17952-29023/merged')
fs.ls('uturuncu-insar/2019-09/D83/int-17959-18134/merged/')

In [ ]:
# Open an image with xarray
xr.__version__

In [ ]:
# Only metadata is read here, which is why it is fast.
# Still, all data on disk is stored as a single chunk
xr.set_options(display_style='html')
da = xr.open_rasterio('s3://uturuncu-insar/2019-09/A76/int-17952-29023/merged/filt_topophase.unw.geo')
print(f'Image size = {da.nbytes / 1e9} Gb')
da

In [ ]:
# It makes more sense to treat bands as separate data variables in an xarray dataset:
da = xr.open_rasterio('s3://uturuncu-insar/2019-09/A76/int-17952-29023/merged/filt_topophase.unw.geo', chunks=dict(band=1,x=12723,y=13440))
da['band'] = ['amplitude','phase']
ds = da.to_dataset(dim='band')
ds

In [ ]:
# We can pull an entire 683 Mb array into memory to work with it,,, hanging.
# NOTE: probably need to explicity launch either a local cluster or kubernetes cluster first
#import hvplot.xarray
#dsp = ds['phase']
#dsp

In [ ]:
# Load a downsampled file:
s3Path = 's3://uturuncu-insar/2019-09/D83/int-17959-18134/merged/filt_topophase.unw.8alks_8rlks.geo'
da = xr.open_rasterio(s3Path)#, chunks=dict(band=1,x=-1,y=-1)) #don't load as dask array since its small
da['band'] = ['amplitude','phase']
ds = da.to_dataset(dim='band')
ds

In [ ]:
# Mark nans, logz 
amp = ds['amplitude']
amp = amp.where(amp != 0)

In [ ]:
type(amp.data)

In [ ]:
img = amp.hvplot.image(geo=True, logz=True, cmap='gray', frame_width=500)
utur = gv.Points([(-67.18, -22.27)]).opts(line_color='r', fill_color=None, marker='^', size=10)
img * utur

In [ ]:
# Plot the wrapped phase
# Mark nans, logz 
phs = ds['phase']
phs = phs.where(phs != 0)

In [ ]:
# Unwrapped phase plot
phs.hvplot.image(geo=True, cmap='bwr', frame_width=500)

In [ ]:
def rewrap(da, wrapRate=6.28):
    func = lambda x: np.remainder(x, wrapRate) / wrapRate
    return xr.apply_ufunc(func, da)


In [ ]:
phs_wrapped = rewrap(phs)

In [ ]:
img = phs_wrapped.hvplot.image(geo=True, cmap='hsv', frame_width=500)
img

In [ ]:
# Plot on basemap with other features

gf = gpd.read_file('apmb.geojson')
# Put these on a map
S,N,W,E = [-25, -20, -72, -65]
tiles = gv.tile_sources.StamenTerrainRetina.options(width=700, height=500).redim.range(Latitude=(S, N), Longitude=(W, E)) 
aoi = gf.hvplot(geo=True, fill_color=None, line_color='orange', hover=False)
utur = gv.Points([(-67.18, -22.27)]).opts(color='r', marker='^', size=10)

# Muted alpha not an option here... need to add a checkboc 
img = phs_wrapped.hvplot.image(geo=True, alpha=0.5, cmap='hsv', frame_width=500, colorbar=False, legend=True)

tiles * img * aoi * utur 